In [1]:
from sklearn.base import BaseEstimator, ClassifierMixin, clone
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

In [64]:
class SimpleMultiClassBoosting(BaseEstimator, ClassifierMixin):
    def __init__(self, base_estimator=None, n_estimators=50):
        self.base_estimator = base_estimator if base_estimator is not None else DecisionTreeClassifier(max_depth=1)
        self.n_estimators = n_estimators
        self.learners = np.empty(n_estimators, dtype=object)
        self.learner_weights = np.empty(n_estimators, dtype=float)
        self.samples_weights = None
        self.label_encoder = LabelEncoder()

    def fit(self, X, y):
        # Convert labels to [0, n_classes-1]
        Y = self.label_encoder.fit_transform(y=y)
        n_classes = len(np.unique(Y))
        indexes = np.arange(X.shape[0])
        # Initialize weights uniformly
        self.samples_weights = (1/X.shape[0]) * np.ones(X.shape[0] ,dtype=float)
        for i in range(self.n_estimators):
            self.learners[i] = self.base_estimator
            training_indexes = np.random.choice(indexes, indexes.shape[0], replace=True, p=self.samples_weights)
            self.learners[i].fit(X[training_indexes], Y[training_indexes])
            # Compute weighted error rate (misclassification rate)
            missClassifiedIndexes = []
            predictions = self.predict(X=X)
            for j in range(X.shape[0]):
                if predictions[j] != y[j]:
                    missClassifiedIndexes.append(j)
            learner_error = (np.sum(self.samples_weights[missClassifiedIndexes])) / np.sum(self.samples_weights)        
            # Compute learner weight using SAMME algorithm
            self.learner_weights[i] = 0.5*np.log((1 - learner_error) / learner_error)
            if learner_error >= 1 - (1 / n_classes):
                self.learner_weights[i] = 0

            # Increase the weights of misclassified samples
            # for index, weigth in enumerate(self.samples_weights):
                # if index in missClassifiedIndexes:
                    # self.samples_weights[index] += self.weigh_increase_ratio
            # self.samples_weights = self.samples_weights / (np.sum(self.samples_weights))       


            for idx in missClassifiedIndexes:
                self.samples_weights[idx] *= np.exp(self.learner_weights[i])

            # Normalize the sample weights
            self.samples_weights /= np.sum(self.samples_weights) 
            # Save the current learner
            # self.learners.append(learner)
            # self.learner_weights.append(learner_weight)

    def predict(self, X):
        # Collect predictions from each learner
        # predictionsOfLearners = []
        # for learner in self.learners:
        #     if learner != None:
        #         predictionsOfLearners.append(learner.predict(X))
        # predictionsOfLearners = np.array(predictionsOfLearners)
        # prediction = np.empty(X.shape[0])        
        # for i in range(X.shape[0]):
        #     labels = np.unique(predictionsOfLearners[:, i])
        #     votes = {label : 0 for label in labels}
        #     for j in range(len(predictionsOfLearners[:, i])):
        #         for label in labels:
        #             if predictionsOfLearners[j, i] == label:
        #                 votes[label] += self.learner_weights[j]
        #     finalPrediction = max(votes, key=votes.get)
        #     prediction[i] = self.label_encoder.inverse_transform(np.array([finalPrediction]))

        # return prediction

        # def predict(self, X):
        predictionsOfLearners = []
        for learner in self.learners:
            if learner is not None:
                predictionsOfLearners.append(learner.predict(X))
        predictionsOfLearners = np.array(predictionsOfLearners)

        prediction = np.zeros(X.shape[0], dtype=int)

        for i in range(X.shape[0]):
            labels = np.unique(predictionsOfLearners[:, i])
            votes = {label: 0 for label in labels}
            for j in range(len(predictionsOfLearners)):
                for label in labels:
                    if predictionsOfLearners[j, i] == label:
                        votes[label] += self.learner_weights[j]
    
            finalPrediction = max(votes, key=votes.get)
            prediction[i] = finalPrediction
        return self.label_encoder.inverse_transform(prediction)


        # labels = np.unique(predictions)
        # votes = {label : 0 for label in labels}
        # for i in range(len(predictions)):
            # for label in labels:
                # if predictions[0][i] == label:
                    # votes[label] += self.learner_weights[i]
        
        # # Weighted vote for each sample's prediction across all learners
        # final_prediction = max(votes, key=votes.get)
        # Final prediction is the one with the highest weighted vote
        # return self.label_encoder.inverse_transform(np.array([final_prediction]))
        # Convert back to original class labels


In [13]:
def fit(self, X, y):
        Y = self.label_encoder.fit_transform(y=y)
        n_classes = len(np.unique(Y))
        indexes = np.arange(X.shape[0])
        self.samples_weights = (1/X.shape[0]) * np.ones(X.shape[0] ,dtype=float)
        for i in range(self.n_estimators):
            self.learners[i] = self.base_estimator
            training_indexes = np.random.choice(indexes, indexes.shape[0], replace=True, p=self.samples_weights)
            self.learners[i].fit(X[training_indexes], Y[training_indexes])
            missClassifiedIndexes = []
            predictions = self.predict(X=X)
            for j in range(X.shape[0]):
                if predictions[j] != y[j]:
                    missClassifiedIndexes.append(j)
            learner_error = (np.sum(self.samples_weights[missClassifiedIndexes])) / np.sum(self.samples_weights)        
            self.learner_weights[i] = 0.5*np.log((1 - learner_error) / learner_error)
            if learner_error >= 1 - (1 / n_classes):
                self.learner_weights[i] = 0
            # Increase the weights of misclassified samples

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]
[1 5 9]


In [4]:
iris = load_iris()
df = pd.DataFrame(data=np.c_[iris['data'], iris['target']], columns=iris['feature_names'] + ['target'])
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(df.iloc[:, :-1].to_numpy(), df.iloc[:, -1].to_numpy(), test_size=0.2, random_state=42)

In [80]:
m = SimpleMultiClassBoosting()
m.fit(X_train, Y_train)

In [81]:
prediction = m.predict(X_test)
prediction

array([1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
       1., 1., 2., 1., 1., 1., 2., 2., 2., 2., 2., 1., 1.])

In [82]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, prediction))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        10
         1.0       0.41      1.00      0.58         9
         2.0       1.00      0.73      0.84        11

    accuracy                           0.57        30
   macro avg       0.47      0.58      0.47        30
weighted avg       0.49      0.57      0.48        30



C:\Users\javadpsk\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\javadpsk\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\javadpsk\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz